## Test the /search endpoints for auxip and cadip

In [4]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

Auxip service: http://rs-server-adgs:8000
CADIP service: http://rs-server-cadip:8000
Catalog service: http://rs-server-catalog:8000


In [5]:

import itertools
import json
import sys
from dataclasses import dataclass
from datetime import timedelta
from urllib.parse import unquote

!pip install iso8601 rfc3339
import iso8601
import rfc3339

In [6]:
@dataclass
class Error:
    """Errors in this tests."""

    endpoint: str
    params: dict
    messages: list[str]

    def __repr__(self):
        sep = "\n  - "
        return (
            f"\nPOST {self.endpoint!r}\n{json.dumps(self.params, indent=2)}\n"
            f"Error(s):{sep}{sep.join(self.messages)}\n"
        )
    
errors: list[Error] = []

def save_error(*args) -> None:
    """Print and save an error"""
    error = Error(*args)
    print(error, file=sys.stderr)
    errors.append(error)


In [7]:
# Collections to search. If None: search all collections.
adgs_collections = None  #["adgs", "adgs2"]
cadip_collections = None
collections = None

def search_and_check(endpoint, in_feature=None, search_properties=[], pagination={}):
    """Call the /search endpoint, check the results, return features."""

    # Build the query parameters
    params = {"query": {}}
    if collections:
        params["collections"] = collections
    for property in search_properties:
        in_value = in_feature.get(property) or in_feature["properties"].get(property)

        if property == "id":
            params["ids"] = [in_value]

        # Date interval start and stop use the input feature date.
        # We need to remove 1 second from the start, and add 1 to the stop.
        elif property == "datetime":
            dt = iso8601.parse_date(in_value)
            start = rfc3339.rfc3339(dt + timedelta(seconds=-1))
            stop = rfc3339.rfc3339(dt + timedelta(seconds=1))
            params[property] = f"{start}/{stop}".replace("+00:00", "Z")

        # query parameters
        else:
            params["query"][property] = {"eq": in_value}

    # Add pagination
    for property, value in pagination.items():
        params[property] = value

    print(f"Call POST {endpoint!r} with params: {list(search_properties) + list(pagination.keys())}")

    # Call the search endpoint, read the returned features
    response = http_session.post(endpoint, json=params)

    if response.status_code != 200:
        save_error(endpoint, params, [f"Status code: {response.status_code}\n{unquote(response.content)}"])
        return

    ret_features = response.json()["features"]
    if not ret_features:
        save_error(endpoint, params, [f"No features returned"])
        return

    # Check that each returned feature property on which we filtered,
    # has the same value than in the input feature.
    messages: list[str] = []
    for property in search_properties:
        for ret_feature in ret_features:
            in_value = in_feature.get(property) or in_feature["properties"].get(property)
            ret_value = ret_feature.get(property) or ret_feature["properties"].get(property)

            if in_value != ret_value:
                messages.append(f"Wrong {property}: {ret_value!r}, expected: {in_value!r}")
                break  # only print error for first wrong feature

    if messages:
        save_error(endpoint, params, messages)
    return ret_features

In [8]:
#############
# Main code #
#############

for service in "auxip", "cadip":

    # For better readability
    auxip = service == "auxip"
    cadip = service == "cadip"

    # Init
    if auxip:
        endpoint = f"{auxip_client.href_adgs}/auxip/search"
        collections = adgs_collections
    elif cadip:
        endpoint = f"{cadip_client.href_cadip}/cadip/search"
        collections = cadip_collections

    # Get all auxip products or cadip sessions
    features = search_and_check(endpoint, pagination={"limit": 10000})

    # We take any existing feature returned by the stations, filter on its properties,
    # and check that the filter was applied.
    feature = features[1]

    # All properties on which we can filter
    properties = ["id", "datetime", "platform"]
    if auxip:
        properties += ["constellation", "product:type"]

    # Test all combinations of n properties
    for length in range(1, len(properties) + 1):
        for search_properties in itertools.combinations(properties, length):
            search_and_check(endpoint, feature, search_properties)

if errors:
    message = "\n## Error message start ##\n"
    for error in errors:
        message += str(error)
    message += "\n## Error message finish ##\n"
    raise RuntimeError(message)

Call POST 'http://rs-server-adgs:8000/auxip/search' with params: ['limit']
Call POST 'http://rs-server-adgs:8000/auxip/search' with params: ['id']
Call POST 'http://rs-server-adgs:8000/auxip/search' with params: ['datetime']
Call POST 'http://rs-server-adgs:8000/auxip/search' with params: ['platform']



POST 'http://rs-server-adgs:8000/auxip/search'
{
  "query": {
    "platform": {
      "eq": "sentinel-1a"
    }
  }
}
Error(s):
  - Status code: 503
{"detail":"General failure: ('Skipping error while searching for adgs ODataV4Search instance', \"400 Client Error: BAD REQUEST for url: http://adgs-station:5000/Products?$filter=Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'AUX_PP2') and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformSerialIdentifier' and att/OData.CSC.StringAttribute/Value eq 'A') and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformShortName' and att/OData.CSC.StringAttribute/Value eq 'sentinel-1')&$orderby=PublicationDate desc&$top=10000&$skip=0&$expand=Attributes\")"}



Call POST 'http://rs-server-adgs:8000/auxip/search' with params: ['constellation']



POST 'http://rs-server-adgs:8000/auxip/search'
{
  "query": {
    "constellation": {
      "eq": "sentinel-1"
    }
  }
}
Error(s):
  - Status code: 503
{"detail":"General failure: ('Skipping error while searching for adgs ODataV4Search instance', \"400 Client Error: BAD REQUEST for url: http://adgs-station:5000/Products?$filter=Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'AUX_PP2') and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformShortName' and att/OData.CSC.StringAttribute/Value eq 'sentinel-1')&$orderby=PublicationDate desc&$top=10000&$skip=0&$expand=Attributes\")"}



Call POST 'http://rs-server-adgs:8000/auxip/search' with params: ['product:type']


...

Call POST 'http://rs-server-cadip:8000/cadip/search' with params: ['datetime', 'platform']



POST 'http://rs-server-cadip:8000/cadip/search'
{
  "query": {
    "platform": {
      "eq": "sentinel-2b"
    }
  },
  "datetime": "2023-11-17T06:15:36Z/2023-11-17T06:15:38Z"
}
Error(s):
  - Status code: 503
{"detail":"General failure: ('Skipping error while searching for sgs_session ODataV4Search instance', \"404 Client Error: NOT FOUND for url: http://cadip-station:5000/Sessions?$filter=Satellite in ('S3A', 'S3B') and PublicationDate gt 2023-11-17T06:15:36.000Z and PublicationDate lt 2023-11-17T06:15:38.000Z&$orderby=PublicationDate desc&$top=10000&$skip=0\")"}



Call POST 'http://rs-server-cadip:8000/cadip/search' with params: ['id', 'datetime', 'platform']



POST 'http://rs-server-cadip:8000/cadip/search'
{
  "query": {
    "platform": {
      "eq": "sentinel-2b"
    }
  },
  "ids": [
    "S2B_20231117033237234567"
  ],
  "datetime": "2023-11-17T06:15:36Z/2023-11-17T06:15:38Z"
}
Error(s):
  - No features returned



RuntimeError: 
## Error message start ##

POST 'http://rs-server-adgs:8000/auxip/search'
{
  "query": {
    "platform": {
      "eq": "sentinel-1a"
    }
  }
}
Error(s):
  - Status code: 503
{"detail":"General failure: ('Skipping error while searching for adgs ODataV4Search instance', \"400 Client Error: BAD REQUEST for url: http://adgs-station:5000/Products?$filter=Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'AUX_PP2') and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformSerialIdentifier' and att/OData.CSC.StringAttribute/Value eq 'A') and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformShortName' and att/OData.CSC.StringAttribute/Value eq 'sentinel-1')&$orderby=PublicationDate desc&$top=10000&$skip=0&$expand=Attributes\")"}

POST 'http://rs-server-adgs:8000/auxip/search'
{
  "query": {
    "constellation": {
      "eq": "sentinel-1"
    }
  }
}
Error(s):
  - Status code: 503
{"det